In [1]:
import requests
import pandas as pd

def obtener_temperatura(lat, lon, fecha_inicio, fecha_fin):
    url = "https://archive-api.open-meteo.com/v1/archive"
    
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": fecha_inicio,
        "end_date": fecha_fin,
        "hourly": "temperature_2m",
        "timezone": "auto"
    }

    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        return {"error": "Error en la respuesta de la API", "codigo": response.status_code}

    data = response.json()
    temperaturas = data.get("hourly", {}).get("temperature_2m", [])
    timestamps = data.get("hourly", {}).get("time", [])

    if not temperaturas or not timestamps:
        return {"error": "No se encontraron datos de temperatura"}

    df = pd.DataFrame({
        "timestamp": timestamps,
        "temperatura_2m": temperaturas
    })

    # --- Indicadores de calidad ---
    calidad = {}
    calidad["total_registros"] = len(df)
    calidad["registros_nulos"] = df["temperatura_2m"].isna().sum()
    calidad["temperatura_min"] = df["temperatura_2m"].min()
    calidad["temperatura_max"] = df["temperatura_2m"].max()

    # Valores extremos para alertar si hay datos sospechosos
    if df["temperatura_2m"].min() < -30 or df["temperatura_2m"].max() > 50:
        calidad["advertencia_valores_extremos"] = True
    else:
        calidad["advertencia_valores_extremos"] = False

    return {"datos": df, "calidad": calidad}

# Ejemplo: Lima (latitud, longitud) y rango de fechas
resultado = obtener_temperatura(-12.0464, -77.0428, "2025-03-25", "2025-03-31")

if "error" in resultado:
    print("Error:", resultado["error"])
else:
    print("Indicadores de calidad:")
    for k, v in resultado["calidad"].items():
        print(f" - {k}: {v}")
    print("\nPrimeras filas de datos:")
    print(resultado["datos"].head())


Indicadores de calidad:
 - total_registros: 168
 - registros_nulos: 0
 - temperatura_min: 19.5
 - temperatura_max: 25.8
 - advertencia_valores_extremos: False

Primeras filas de datos:
          timestamp  temperatura_2m
0  2025-03-25T00:00            20.6
1  2025-03-25T01:00            20.1
2  2025-03-25T02:00            20.1
3  2025-03-25T03:00            20.0
4  2025-03-25T04:00            19.8


# JUSTIFICACION DE LOS INDICADORES DE CALIDAD ###

### total_registros: 
Esto asegura tener la cantidad de registros esperados
### registros_nulos:
Esto asegura que no falte ningun registro
### advertencia_valores_extremos: 
Esto ayuda a identificar errores si los valores estan fuera de un rango razonable